In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
element_score = pd.read_csv(
    "/Users/kwatme/Downloads/gsea/benchmark4.gene.list.GSEA.txt",
    sep="\t",
    usecols=("gene.list", "gene.scores"),
    index_col=0,
    squeeze=True,
)

In [ ]:
set_elements = kraft.read_gmt("/Users/kwatme/Downloads/gsea/c2.all.v5.1.symbols.gmt")

In [ ]:
set_elements_ = set_elements["COLLER_MYC_TARGETS_UP"]

method = "rank cdf js"

print(kraft.compute_set_enrichment(element_score, set_elements_, method=method))

print(
    kraft.compute_set_enrichment(
        pd.Series(element_score.values[::-1], element_score.index),
        set_elements_,
        method=method,
    )
)

In [ ]:
for sides in (
    #     ("-"),
    #     ("+",),
    ("-", "+"),
):

    for n_element in (6,):

        assert n_element % 2 == 0

        element_score = np.random.normal(size=n_element // 2)

        element_score = pd.Series(
            np.sort(np.concatenate((element_score, -element_score))),
            index=("Element{}".format(i) for i in range(n_element)),
        )

        for n_skip in (0,):

            if n_skip < 1:

                n_skip = int(n_skip * n_element)

            for n_top in (2,):

                if n_top < 1:

                    n_top = int(n_top * n_element)

                for rate_top_hit in (1,):

                    if rate_top_hit < 1:

                        rate_top_hit = int(rate_top_hit * n_top)

                    print("=" * 80)

                    print(n_element, n_skip, n_top, rate_top_hit)

                    print("=" * 80)

                    set_elements_ = []

                    if "-" in sides:

                        set_elements_ += list(
                            element_score.index[n_skip : n_skip + n_top : rate_top_hit]
                        )

                    if "+" in sides:

                        set_elements_ += list(
                            element_score.index[
                                -(n_skip + 1) : -(n_skip + 1 + n_top) : -rate_top_hit
                            ]
                        )[::-1]

                    print(set_elements_)

                    enrichment = kraft.compute_set_enrichment(
                        element_score, set_elements_, power=0, plot=True
                    )

                    print(enrichment)

In [ ]:
n = 10

a = np.arange(1, n) / n

b = a[::-1]

c = (a + b) / 2

jsh = a * np.log(a / c)

jsm = b * np.log(b / c)

js = jsh + jsm

kraft.plot_plotly_figure(
    {"data": [{"y": a}, {"y": b}, {"y": c}, {"y": jsh}, {"y": jsm}, {"y": js},]}
)